# Consulta Postgres

In [158]:
#!pip install pandas
#!pip install nltk
#!pip install psycopg2
#!pip install sqlalchemy
#!pip install flask

In [161]:
# imports iniciais
import os
import nltk
import pandas as pd
import pandas.io.sql as psql
import pandas as pd

In [99]:
#nltk.download()

# Assunto

In [100]:
df = pd.read_csv('dados_teste.csv',delimiter='|')

In [101]:
df.describe()

,numero
count,400.000000
mean,4721.890000
std,2818.420545
min,24.000000
25%,2219.500000
50%,4693.500000
75%,7108.750000
max,9995.000000


In [102]:
df.count()

nome        400
empresa     400
endereco    400
data1       400
data2       400
numero      400
dtype: int64

In [103]:
df.head()

,nome,empresa,endereco,data1,data2,numero
0,Anjolie Q. Campbell,Luctus Company,"P.O. Box 974, 4866 Integer Av.",2018-10-23,2018-11-03,5796
1,Deanna F. Burnett,Quisque Nonummy Consulting,504-1917 Urna Avenue,2018-12-01,2020-06-10,7320
2,Cole Z. Travis,Malesuada Fames Ac Associates,"P.O. Box 627, 3320 Aliquam, Road",2020-08-15,2020-09-02,7639
3,Sonya Y. Mckinney,Mauris Non LLP,Ap #935-8507 Pede Rd.,2018-10-13,2019-12-04,82
4,Logan V. Holmes,Leo Cras Foundation,Ap #235-1628 Consectetuer Rd.,2019-07-18,2019-02-03,6218


In [145]:
import string
from nltk.corpus import stopwords
portuguese_stops = set(stopwords.words('portuguese'))
ignore = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']','``',':','http','html','//members']
#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
#from nltk.stem import RegexpStemmer
from nltk.stem import SnowballStemmer
# Cria o Stemmer
stemmer = SnowballStemmer('english')
def preprocess_text(content):
    content.translate(str.maketrans('', '', string.punctuation))
    word_tokens = nltk.word_tokenize(content)
    #print(len(word_tokens),word_tokens)
    word_tokens2 = [w.lower() for w in word_tokens if w not in portuguese_stops]
    #print(len(word_tokens2),word_tokens2)
    word_tokens3 = [w for w in word_tokens2 if w not in ignore]
    #print(len(word_tokens3),word_tokens3)
    word_tokens4 = [stemmer.stem(w) for w in word_tokens3]
    #print(len(word_tokens4),word_tokens4)
    #return word_tokens4
    return ' '.join(word_tokens4)

In [146]:
teste = 'Teste de sentença para tokenizar'

In [147]:
preprocess_text(teste)

'test sentença tokenizar'

In [148]:
df['nome'] = df['nome'].apply(preprocess_text)

In [149]:
#!pip install gensim
#!pip install sklearn
#!pip install jieba

In [150]:
from gensim import corpora, models, similarities

In [151]:
texts = df['nome']
print(nomes)

0      anjol q. campbel
1      deann f. burnett
2           col z. trav
3      soni y. mckinney
4         logan v. holm
             ...       
395    olga i. thornton
396         xen k. stok
397     cassidi k. pott
398        kyle p. robl
399      jarrod n. robl
Name: nome, Length: 400, dtype: object


In [152]:
import nltk 
import re
import math

import pandas as pd

from collections import Counter
from unicodedata import normalize
from nltk import ngrams

#Regex para encontrar tokens
REGEX_WORD = re.compile(r'\w+')
#Numero de tokens em sequencia
N_GRAM_TOKEN = 3

#Faz a normalizacao do texto removendo espacos a mais e tirando acentos
def text_normalizer(src):
    return re.sub('\s+', ' ',
                normalize('NFKD', src)
                   .encode('ASCII','ignore')
                   .decode('ASCII')
           ).lower().strip()

#Faz o calculo de similaridade baseada no coseno
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        coef = float(numerator) / denominator
        if coef > 1:
            coef = 1
        return coef

#Monta o vetor de frequencia da sentenca
def sentence_to_vector(text, use_text_bigram):
    words = REGEX_WORD.findall(text)
    accumulator = []
    for n in range(1,N_GRAM_TOKEN):
        gramas = ngrams(words, n)
        for grama in gramas:
            accumulator.append(str(grama))

    if use_text_bigram:
        pairs = get_text_bigrams(text)
        for pair in pairs:
            accumulator.append(pair)

    return Counter(accumulator)

#Obtem a similaridade entre duas sentencas
def get_sentence_similarity(sentence1, sentence2, use_text_bigram=False):
    vector1 = sentence_to_vector(text_normalizer(sentence1), use_text_bigram)
    vector2 = sentence_to_vector(text_normalizer(sentence2), use_text_bigram)
    return cosine_similarity(vector1, vector2)

#Metodo de gerar bigramas de uma string
def get_text_bigrams(src):
    s = src.lower()
    return [s[i:i+2] for i in range(len(s) - 1)]

if __name__ == "__main__":
    w1 = 'COMERCIAL CASA DOS FRIOS - USAR LICINIO DIAS'
    words = [
        'ARES DOS ANDES - EXPORTACAO & IMPORTACAO LTDA', 
        'ADEGA DOS TRES IMPORTADORA', 
        'BODEGAS DE LOS ANDES COMERCIO DE VINHOS LTDA', 
        'ALL WINE IMPORTADORA'
    ]

    print('Busca: ' + w1)

    #Nivel de aceite (40%)
    cutoff = 0.40
    #Sentenças similares
    result = []

    for w2 in words:
        print('\nCosine Sentence --- ' + w2)

        #Calculo usando similaridade do coseno com apenas tokens
        similarity_sentence = get_sentence_similarity(w1, w2)
        print('\tSimilarity sentence: ' + str(similarity_sentence))

        #Calculo usando similaridade do coseno com tokens e com ngramas do texto
        similarity_sentence_text_bigram = get_sentence_similarity(w1, w2, use_text_bigram=True)
        print('\tSimilarity sentence: ' + str(similarity_sentence_text_bigram))

        if similarity_sentence >= cutoff:
            result.append((w2, similarity_sentence))

    print('\nResultado:')
    #Exibe resultados
    for data in result:
        print(data)

Busca: COMERCIAL CASA DOS FRIOS - USAR LICINIO DIAS

Cosine Sentence --- ARES DOS ANDES - EXPORTACAO & IMPORTACAO LTDA
	Similarity sentence: 0.08362420100070908
	Similarity sentence: 0.26518576139191

Cosine Sentence --- ADEGA DOS TRES IMPORTADORA
	Similarity sentence: 0.10482848367219183
	Similarity sentence: 0.223606797749979

Cosine Sentence --- BODEGAS DE LOS ANDES COMERCIO DE VINHOS LTDA
	Similarity sentence: 0.0
	Similarity sentence: 0.39317854974639244

Cosine Sentence --- ALL WINE IMPORTADORA
	Similarity sentence: 0.0
	Similarity sentence: 0.09245003270420486

Resultado:


In [153]:
texts.count()

400

In [155]:
max_sim = 0
most_similar = ''
test = 'John da silva'
for w in texts:
    sim = get_sentence_similarity(test, w, use_text_bigram=True)
    if sim > max_sim:
        max_sim = sim
        most_similar = w
        print(str(sim)+' '+w)
print ('Mais similar a ' + test + ' = ' + most_similar)

0.05423261445466404 anjol q. campbel
0.058823529411764705 logan v. holm
0.06262242910851494 walt d. pac
0.1143323900950059 joshu i. oneil
0.11764705882352941 ivan i. buckn
0.12126781251816648 ivan f. hobb
0.15512630699850574 lillian h. dixon
0.16269784336399212 alan x. dalton
0.16692446522239712 jason w. daniel
0.17647058823529413 elain y savag
0.18786728732554484 sil q. vang
0.21693045781865616 josiah f. dawson
Mais similar a John da silva = josiah f. dawson


In [159]:
!pip install spacy

In [163]:
import spacy
nlp = spacy.load('pt')

OSError: [E050] Can't find model 'pt'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [155]:
import spacy
nlp = spacy.load('pt')
 
doc1 = nlp(u'Hello this is document similarity calculation')
doc2 = nlp(u'Hello this is python similarity calculation')
doc3 = nlp(u'Hi there')
 
print (doc1.similarity(doc2)) 
print (doc2.similarity(doc3)) 
print (doc1.similarity(doc3))  
 
max_sim = 0
most_similar = ''
test = 'John da silva'
for w in texts:
    sim = get_sentence_similarity(test, w, use_text_bigram=True)
    if sim > max_sim:
        max_sim = sim
        most_similar = w
        print(str(sim)+' '+w)
print ('Mais similar a ' + test + ' = ' + most_similar)

0.05423261445466404 anjol q. campbel
0.058823529411764705 logan v. holm
0.06262242910851494 walt d. pac
0.1143323900950059 joshu i. oneil
0.11764705882352941 ivan i. buckn
0.12126781251816648 ivan f. hobb
0.15512630699850574 lillian h. dixon
0.16269784336399212 alan x. dalton
0.16692446522239712 jason w. daniel
0.17647058823529413 elain y savag
0.18786728732554484 sil q. vang
0.21693045781865616 josiah f. dawson
Mais similar a John da silva = josiah f. dawson


In [156]:
!pip install flask